In [8]:
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
import nltk
import re
import string
import numpy as np

import pandas as pd
from fuzzywuzzy import fuzz

df = pd.read_csv('./scraper/archive/react-useeffect.csv', names=["POSTID", "POSTLINK", "POSTTITLE", "POSTBODY", "POSTDATE", "POSTVOTECOUNTS", "COMMENTID", "COMMENTSCORE", "COMMENTUSERNAME", "COMMENTTEXT", "COMMENTDATETIME", "ANSWERID", "ANSWERTEXT", "ANSWERBODY", "ANSWERDATETIME", "ANSWERVOTECOUNTS", "ANSWERCMTID", "ANSWERCMTTEXT", "ANSWERCMTBODY", "ANSWERCMTDATETIME", "ANSWERCMTVOTECOUNTS" , "TYPE"])
query = 'Infinite rerender'

display(df.head())

,POSTID,POSTLINK,POSTTITLE,POSTBODY,POSTDATE,POSTVOTECOUNTS,COMMENTID,COMMENTSCORE,COMMENTUSERNAME,COMMENTTEXT,...,ANSWERTEXT,ANSWERBODY,ANSWERDATETIME,ANSWERVOTECOUNTS,ANSWERCMTID,ANSWERCMTTEXT,ANSWERCMTBODY,ANSWERCMTDATETIME,ANSWERCMTVOTECOUNTS,TYPE
0,75130904,https://stackoverflow.com/questions/75130904/h...,How to avoid a small delay in scrolling on rea...,\n \n<p>I have a component that...,"('2023-01-16 06:36:31Z',)","('2',)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,post
1,75130904,https://stackoverflow.com/questions/75130904/h...,How to avoid a small delay in scrolling on rea...,\n \n<p>I have a component that...,"('2023-01-16 06:36:31Z',)","('2',)",NaN,NaN,NaN,NaN,...,"It sounds like you may need: useLayoutEffect, ...","('\n<p>It sounds like you may need: <a href=""...","2023-01-16 07:38:44Z, License: CC BY-SA 4.0","('2',)",NaN,NaN,NaN,NaN,NaN,answer
2,75130904,https://stackoverflow.com/questions/75130904/h...,How to avoid a small delay in scrolling on rea...,\n \n<p>I have a component that...,"('2023-01-16 06:36:31Z',)","('2',)",NaN,NaN,NaN,NaN,...,"It sounds like you may need: useLayoutEffect, ...","('\n<p>It sounds like you may need: <a href=""...","2023-01-16 07:38:44Z, License: CC BY-SA 4.0","('2',)","('132582008',)",it works! thank you! never used useLayoutEffec...,('\n \n <span cl...,14 hours ago,"('',)",answer_comment
3,75122338,https://stackoverflow.com/questions/75122338/r...,React useEffect called twice even with strict ...,\n \n<p>I'm new to React and am...,"('2023-01-15 01:01:27Z',)","('1',)",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,post
4,75122338,https://stackoverflow.com/questions/75122338/r...,React useEffect called twice even with strict ...,\n \n<p>I'm new to React and am...,"('2023-01-15 01:01:27Z',)","('1',)",comment-132566773,NaN,segFault,"You might just be missing a dependency array, ...",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,post_comment


In [9]:
ALL_COLS = ["POSTID", "POSTLINK", "POSTTITLE", "POSTBODY", "POSTDATE", "POSTVOTECOUNTS", "COMMENTID", "COMMENTSCORE", "COMMENTUSERNAME", "COMMENTTEXT", "COMMENTDATETIME", "ANSWERID", "ANSWERTEXT", "ANSWERBODY", "ANSWERDATETIME", "ANSWERVOTECOUNTS", "ANSWERCMTID", "ANSWERCMTTEXT", "ANSWERCMTBODY", "ANSWERCMTDATETIME", "ANSWERCMTVOTECOUNTS" , "TYPE"]

dPOST  = pd.DataFrame(columns=ALL_COLS)
dPOST = df.copy();
dPOST = dPOST.drop(columns=['COMMENTID', 'COMMENTSCORE', 'COMMENTUSERNAME', 'COMMENTTEXT', 'COMMENTDATETIME', 'ANSWERID', 'ANSWERTEXT', 'ANSWERBODY', 'ANSWERDATETIME', 'ANSWERVOTECOUNTS', 'ANSWERCMTID', 'ANSWERCMTTEXT', 'ANSWERCMTBODY', 'ANSWERCMTDATETIME', 'ANSWERCMTVOTECOUNTS'])
# drop if type is not post
dPOST = dPOST[dPOST['TYPE'] == 'post']
display(dPOST)

dfPC = pd.DataFrame(columns=ALL_COLS)
dfPC = df.copy();
dfPC = dfPC.drop(columns=['ANSWERID', 'ANSWERTEXT', 'ANSWERBODY', 'ANSWERDATETIME', 'ANSWERVOTECOUNTS', 'ANSWERCMTID', 'ANSWERCMTTEXT', 'ANSWERCMTBODY', 'ANSWERCMTDATETIME', 'ANSWERCMTVOTECOUNTS'])
# drop if type is not post
dfPC = dfPC[dfPC['TYPE'] == 'post_comment']
display(dfPC)

dfA = pd.DataFrame(columns=ALL_COLS)
dfA = df.copy();
dfA = dfA.drop(columns=['POSTLINK', 'POSTBODY', 'POSTDATE', 'POSTVOTECOUNTS', 'COMMENTID', 'COMMENTSCORE', 'COMMENTUSERNAME', 'COMMENTTEXT', 'COMMENTDATETIME', 'ANSWERCMTID', 'ANSWERCMTTEXT', 'ANSWERCMTBODY', 'ANSWERCMTDATETIME', 'ANSWERCMTVOTECOUNTS'])

dfAAC = dfA[dfA['TYPE'] == 'answer']
display(dfAAC)

dFAC = pd.DataFrame(columns=ALL_COLS)
dFAC = df.copy();
dFAC = dFAC.drop(columns=['POSTLINK', 'POSTBODY', 'POSTDATE', 'POSTVOTECOUNTS', 'COMMENTID', 'COMMENTSCORE', 'COMMENTUSERNAME', 'COMMENTTEXT', 'COMMENTDATETIME', 'ANSWERTEXT', 'ANSWERBODY', 'ANSWERDATETIME', 'ANSWERVOTECOUNTS'])
dFACT = dFAC[dFAC['TYPE'] == 'answer_comment']
display(dFACT)

,POSTID,POSTLINK,POSTTITLE,POSTBODY,POSTDATE,POSTVOTECOUNTS,TYPE
0,75130904,https://stackoverflow.com/questions/75130904/h...,How to avoid a small delay in scrolling on rea...,\n \n<p>I have a component that...,"('2023-01-16 06:36:31Z',)","('2',)",post
3,75122338,https://stackoverflow.com/questions/75122338/r...,React useEffect called twice even with strict ...,\n \n<p>I'm new to React and am...,"('2023-01-15 01:01:27Z',)","('1',)",post
11,75118187,https://stackoverflow.com/questions/75118187/r...,React useEffect not triggered on element clien...,\n \n<p>I have a component with...,"('2023-01-14 13:12:48Z',)","('1',)",post
17,75114654,https://stackoverflow.com/questions/75114654/s...,socket.io fires event twice in react useEffect...,\n <div>\n <asid...,"('2023-01-13 21:31:52Z',)","('0',)",post
22,75108382,https://stackoverflow.com/questions/75108382/r...,React useEffect and onClick button do not work,\n \n<p>App.js</p>\n<pre class=...,"('2023-01-13 11:06:56Z',)","('1',)",post
...,...,...,...,...,...,...,...
3310,55838263,https://stackoverflow.com/questions/55838263/r...,React useEffect - Run on Mount Only,"\n \n<p>Recently, I updated an ...","('2019-04-24 20:42:01Z',)","('1',)",post
3311,55286732,https://stackoverflow.com/questions/55286732/r...,React useEffect weird behavior,\n \n<p>I have a form component...,"('2019-03-21 18:09:58Z',)","('0',)",post
3314,55011667,https://stackoverflow.com/questions/55011667/r...,React useEffect hook referencing incorrect values,\n \n<p>I am working on creatin...,"('2019-03-05 21:12:58Z',)","('2',)",post
3321,54010272,https://stackoverflow.com/questions/54010272/w...,when using react useEffect my app is crashing,\n \n<p>i m changing a class co...,"('2019-01-02 16:59:40Z',)","('6',)",post


,POSTID,POSTLINK,POSTTITLE,POSTBODY,POSTDATE,POSTVOTECOUNTS,COMMENTID,COMMENTSCORE,COMMENTUSERNAME,COMMENTTEXT,COMMENTDATETIME,TYPE
4,75122338,https://stackoverflow.com/questions/75122338/r...,React useEffect called twice even with strict ...,\n \n<p>I'm new to React and am...,"('2023-01-15 01:01:27Z',)","('1',)",comment-132566773,NaN,segFault,"You might just be missing a dependency array, ...","2023-01-15 01:10:53Z, License: CC BY-SA 4.0",post_comment
12,75118187,https://stackoverflow.com/questions/75118187/r...,React useEffect not triggered on element clien...,\n \n<p>I have a component with...,"('2023-01-14 13:12:48Z',)","('1',)",comment-132559365,NaN,Sergey Sosunov,Does this answer your question? Detect element...,"2023-01-14 13:19:20Z, License: CC BY-SA 4.0",post_comment
13,75118187,https://stackoverflow.com/questions/75118187/r...,React useEffect not triggered on element clien...,\n \n<p>I have a component with...,"('2023-01-14 13:12:48Z',)","('1',)",comment-132559388,NaN,Sergey Sosunov,Or this one useEffect without deps array. You ...,"2023-01-14 13:22:20Z, License: CC BY-SA 4.0",post_comment
14,75118187,https://stackoverflow.com/questions/75118187/r...,React useEffect not triggered on element clien...,\n \n<p>I have a component with...,"('2023-01-14 13:12:48Z',)","('1',)",comment-132559452,NaN,Raz Akbari,@SergeySosunov thank you for your comment. Do ...,"2023-01-14 13:29:16Z, License: CC BY-SA 4.0",post_comment
15,75118187,https://stackoverflow.com/questions/75118187/r...,React useEffect not triggered on element clien...,\n \n<p>I have a component with...,"('2023-01-14 13:12:48Z',)","('1',)",comment-132559805,NaN,Eduardo Motta de Moraes,useRef doesn’t notify you when its content cha...,"2023-01-14 13:59:59Z, License: CC BY-SA 4.0",post_comment
...,...,...,...,...,...,...,...,...,...,...,...,...
3309,55936937,https://stackoverflow.com/questions/55936937/a...,arrow-body-style error on react useEffect method,\n \n<p>Here are my codes using...,"('2019-05-01 13:21:17Z',)","('3',)",comment-98526716,NaN,Dave Newton,The function you pass to useEffect has curly b...,"2019-05-01 13:28:52Z, License: CC BY-SA 4.0",post_comment
3312,55286732,https://stackoverflow.com/questions/55286732/r...,React useEffect weird behavior,\n \n<p>I have a form component...,"('2019-03-21 18:09:58Z',)","('0',)",comment-97304600,NaN,Shubham Khatri,can you show your form component,"2019-03-21 18:57:38Z, License: CC BY-SA 4.0",post_comment
3313,55286732,https://stackoverflow.com/questions/55286732/r...,React useEffect weird behavior,\n \n<p>I have a form component...,"('2019-03-21 18:09:58Z',)","('0',)",comment-97306822,NaN,Olivier Boissé,The ParentComponent seems to be ok as you use ...,"2019-03-21 20:27:07Z, License: CC BY-SA 4.0",post_comment
3322,54010272,https://stackoverflow.com/questions/54010272/w...,when using react useEffect my app is crashing,\n \n<p>i m changing a class co...,"('2019-01-02 16:59:40Z',)","('6',)",comment-94858132,NaN,Omar,whats the error message,"2019-01-02 17:34:36Z, License: CC BY-SA 4.0",post_comment


,POSTID,POSTTITLE,ANSWERID,ANSWERTEXT,ANSWERBODY,ANSWERDATETIME,ANSWERVOTECOUNTS,TYPE
1,75130904,How to avoid a small delay in scrolling on rea...,75131129.0,"It sounds like you may need: useLayoutEffect, ...","('\n<p>It sounds like you may need: <a href=""...","2023-01-16 07:38:44Z, License: CC BY-SA 4.0","('2',)",answer
5,75122338,React useEffect called twice even with strict ...,75122385.0,The issue here is that the useEffect hook is m...,('\n<p>The issue here is that the <code>useEff...,"2023-01-15 02:00:46Z, License: CC BY-SA 4.0","('4',)",answer
19,75114654,socket.io fires event twice in react useEffect...,75114744.0,"""the only way this useEffect hook render once ...","('\n<p>""the only way this useEffect hook rende...","2023-01-14 06:16:44Z, License: CC BY-SA 4.0","('1',)",answer
24,75108382,React useEffect and onClick button do not work,75108557.0,"you need to send child components personData, ...",('\n<p>you need to send child components perso...,"2023-01-13 13:45:43Z, License: CC BY-SA 4.0","('0',)",answer
32,75100399,Listen localStorage in react(useEffect),75100694.0,Based on this answer Listen for changes with l...,"('\n<p>Based on this answer <a href=""https://s...","2023-01-13 06:21:39Z, License: CC BY-SA 4.0","('1',)",answer
...,...,...,...,...,...,...,...,...
3285,56382133,React: useEffect performance issue on Android ...,56389865.0,useLayoutEffect is synonym for componentDidMou...,('\n<p>useLayoutEffect is synonym for componen...,"2019-05-31 10:45:46Z, License: CC BY-SA 4.0","('0',)",answer
3291,56258993,React useEffect and iterating over array using...,56259027.0,That a look at this answer and also what I fou...,"('\n<p>That a look at <a href=""https://stackov...","2019-05-22 14:30:52Z, License: CC BY-SA 4.0","('0',)",answer
3300,56047032,React useEffect - Why property from closure ob...,56047450.0,"If the button click is throwing ""Not implement...","('\n<p>If the button click is throwing ""Not im...","2019-05-09 12:30:57Z, License: CC BY-SA 4.0","('0',)",answer
3315,55011667,React useEffect hook referencing incorrect values,55011913.0,The main problem comes from the way you call u...,('\n<p>The main problem comes from the way you...,"2019-03-05 21:36:14Z, License: CC BY-SA 4.0","('3',)",answer


,POSTID,POSTTITLE,ANSWERID,ANSWERCMTID,ANSWERCMTTEXT,ANSWERCMTBODY,ANSWERCMTDATETIME,ANSWERCMTVOTECOUNTS,TYPE
2,75130904,How to avoid a small delay in scrolling on rea...,75131129.0,"('132582008',)",it works! thank you! never used useLayoutEffec...,('\n \n <span cl...,14 hours ago,"('',)",answer_comment
6,75122338,React useEffect called twice even with strict ...,75122385.0,"('132567007',)","updating the dependency may resolve the issue,...",('\n \n <span cl...,yesterday,"('',)",answer_comment
7,75122338,React useEffect called twice even with strict ...,75122385.0,"('132567065',)",@Shah fetchWeekly is an async function and it'...,('\n \n <span cl...,yesterday,"('1',)",answer_comment
8,75122338,React useEffect called twice even with strict ...,75122385.0,"('132567629',)","Yes, I see what you’re saying. I should have b...",('\n \n <span cl...,yesterday,"('',)",answer_comment
9,75122338,React useEffect called twice even with strict ...,75122385.0,"('132567656',)",I noticed your GraphQL layer may not handling ...,('\n \n <span cl...,yesterday,"('',)",answer_comment
...,...,...,...,...,...,...,...,...,...
3318,55011667,React useEffect hook referencing incorrect values,55011913.0,"('96778735',)",Thanks for the thought @Strebler. Unfortunatel...,('\n \n <span cl...,"Mar 5, 2019 at 21:38","('',)",answer_comment
3319,55011667,React useEffect hook referencing incorrect values,55011913.0,"('96778880',)",props.onEnd does actually update since you rec...,('\n \n <span cl...,"Mar 5, 2019 at 21:44","('',)",answer_comment
3320,55011667,React useEffect hook referencing incorrect values,55011913.0,"('96779386',)",@Strebler The if (!sliderElement.current) bloc...,('\n \n <span cl...,"Mar 5, 2019 at 22:06","('',)",answer_comment
3325,54010272,when using react useEffect my app is crashing,54021675.0,"('96213635',)",Thank you! The second answer helped me with my...,('\n \n <span cl...,"Feb 15, 2019 at 17:07","('',)",answer_comment


### Data Explanatory

1. Plot nan values
2. Plot distirbution of scores
3. Plot word average occurence (To find word occurence in tech related text)
4. Format time

### String Similarity to match Post Title with the Query (Idea A)
1. Partial Ratio & Token Sort Ratio on Post Title

### String Similarity to match Answers with the Query (Idea B)
1. Partial Ratio & Token Sort Ratio on Answers 

### Group all the answers based on which idea chosen and preprocess 
1. Stemming
2. ...

### Rank them based on all comments gathered 

### Reorder the post based on average sentiment of the answers in the post (For both ideas)
1. Can compare this with both ideas

### Reorder the sequences based on time (Time analysis)

In [10]:
def calPartialRatio (str1, str2):
    return fuzz.partial_ratio(str1, str2)
  
def calTokenSortRatio (str1, str2):
    return fuzz.token_sort_ratio(str1, str2)
  
class GroupedComments: 
    def __init__(self, title, post, post_comments, answers, answer_comments): 
        self.title = title
        self.post = post
        self.post_comments = post_comments
        self.answers = answers
        self.answer_comments = answer_comments
        

def grouping(titleArr): 
    grouped = []
    for title in titleArr:
        post = dPOST[dPOST['POSTTITLE'].str.contains(title)]
        post_comments = dfPC[dfPC['POSTTITLE'].str.contains(title)]
        answers = dfAAC[dfAAC['POSTTITLE'].str.contains(title)]
        answer_comments = dFACT[dFACT['POSTTITLE'].str.contains(title)]
        grouped.append(GroupedComments(title, post, post_comments, answers, answer_comments))
    return grouped 

In [12]:
# get all unique values in column 3
unique_values = df['POSTTITLE'].unique()
print("There are a total of {} unique titles".format(len(unique_values)))

# create another dataframe to store the results
df2 = pd.DataFrame(columns=['postId', 'title', 'partial_ratio', 'token_sort_ratio'])

# calculate the partial ratio for each unique value
for unique_value in unique_values:
    # find the postId of the unique value
    postId = df[df['POSTTITLE'] == unique_value]['POSTID'].tolist()[0]
    df2 = pd.concat([df2, pd.DataFrame({'postId': postId , 'title': unique_value, 'partial_ratio': calPartialRatio(unique_value, query), 'token_sort_ratio': calTokenSortRatio(unique_value, query)}, index=[0])], )

Top10MatchesTS = df2.sort_values(by=['token_sort_ratio'], ascending=False).head(10)
Top10MatchesPR = df2.sort_values(by=['partial_ratio'], ascending=False).head(10)

# add a new column to store the average of the two ratios
Top10MatchesTS['average'] = (Top10MatchesTS['token_sort_ratio'] + Top10MatchesTS['partial_ratio']) / 2
Top10MatchesPR['average'] = (Top10MatchesPR['token_sort_ratio'] + Top10MatchesPR['partial_ratio']) / 2

# sort the dataframe by the average column
Top10MathesMerged = pd.concat([Top10MatchesTS, Top10MatchesPR]).drop_duplicates().sort_values(by=['average'], ascending=False).head(10)

display(Top10MatchesTS)

# bag of words approach (https://stackoverflow.com/questions/51623595/better-approach-than-fuzzywuzzy)
def myScore(lst1, lst2):
    # calculate score for comparing lists of words
    c = sum(el in lst1 for el in lst2)
    if (len(lst1) == 0 or len(lst2) == 0):
        retval = 0.0
    else:
        retval = 0.5 * (c/len(lst1) + c/len(lst2))
    
    return retval
  
query_tokenized = nltk.word_tokenize(query)

# create another dataframe to store the results
df3 = pd.DataFrame(columns=['postId', 'title', 'score'])
 
# calculate the score for each unique value
for unique_value in unique_values:
    # find the postId of the unique value
    postId = df[df['POSTTITLE'] == unique_value]['POSTID'].tolist()[0]
    df3 = pd.concat([df3, pd.DataFrame({'postId': postId , 'title': unique_value, 'score': myScore(nltk.word_tokenize(unique_value), query_tokenized)}, index=[0])], )
     
Top10MatchesBOW = df3.sort_values(by=['score'], ascending=False).head(10)
display(Top10MatchesBOW)

There are a total of 659 unique titles


,postId,title,partial_ratio,token_sort_ratio,average
0,73534338,React UseEffect render infinite loop,53,64,58.5
0,59449940,React useEffect() Infinite loop,60,57,58.5
0,59146735,React useEffect infinite loop,53,57,55.0
0,70110116,React useEffect does infinite loop,53,51,52.0
0,68846124,React useEffect stop infinite loop,53,51,52.0
0,72751014,React useEffect infinite loop cause,47,50,48.5
0,62568196,React useEffect not rerendering,67,50,58.5
0,69038890,React useEffect caused infinite loop,53,49,51.0
0,73364048,React UseEffect Not Rendering,47,48,47.5
0,58557877,React useEffect hook infinity loop,47,47,47.0


,postId,title,score
0,59449940,React useEffect() Infinite loop,0.333333
0,70257707,React useEffect Infinite loop when fetching fr...,0.312500
0,59659279,React useEffect() Infinite loop with firebase ...,0.300000
0,69392525,React useEffect Infinite Loop even though depe...,0.300000
0,73224421,Infinite loop error in react useeffect when us...,0.300000
0,73464053,React useEffect inside parent component doesnt...,0.285714
0,75130904,How to avoid a small delay in scrolling on rea...,0.000000
0,65373481,No response from Fetch API in React useEffect ...,0.000000
0,65370843,React useEffect not firing and Layout doesn't ...,0.000000
0,59725886,Scrolling to a position in an element on page ...,0.000000


In [270]:
allcommentsGrouped = grouping(Top10MatchesTS['title'].tolist())
print(allcommentsGrouped[1].title)

How to append json in Python List


## Now we have all of the infos in hand for the top 10 post, let's group them and join all the texts together for each column

In [271]:
allCommentsGroupedCleanedDict = {}
for idx, item in enumerate(allcommentsGrouped):
    comments = []
    answers = [] 
    answers_cmts = []
    
    for (index, comment) in item.post_comments['COMMENTTEXT'].iteritems():
      comment_idx = item.post_comments[item.post_comments['COMMENTTEXT'] == comment]['COMMENTID'].tolist()[0]
      comment_votes = item.post_comments[item.post_comments['COMMENTTEXT'] == comment]['COMMENTSCORE'].tolist()[0]
      comment_dt = item.post_comments[item.post_comments['COMMENTTEXT'] == comment]['COMMENTDATETIME'].tolist()[0]
      if str(comment) != 'nan':
        comments.append({
          "comment": comment, 
          "id": comment_idx,
          "votes": comment_votes,
          "datetime": comment_dt
        })
    comments_joined = ' '.join([str(elem['comment']) for elem in comments])

    for (index, answer) in item.answers['ANSWERTEXT'].iteritems():
      answer_idx = item.answers[item.answers['ANSWERTEXT'] == answer]['ANSWERID'].tolist()[0]
      answer_votes = item.answers[item.answers['ANSWERTEXT'] == answer]['ANSWERVOTECOUNTS'].tolist()[0]
      answer_dt = item.answers[item.answers['ANSWERTEXT'] == answer]['ANSWERDATETIME'].tolist()[0]
      if str(answer) != 'nan':
        answers.append({
          "answer": answer,
          "id": answer_idx,
          "votes": answer_votes,
          "datetime": answer_dt
        })
        
    answers_joined = ' '.join([str(elem['answer']) for elem in answers])
    
    for (index, answer_cmt) in item.answer_comments['ANSWERCMTTEXT'].iteritems():
      answer_cmt_idx = item.answer_comments[item.answer_comments['ANSWERCMTTEXT'] == answer_cmt]['ANSWERCMTID'].tolist()[0]
      answer_cmt_votes = item.answer_comments[item.answer_comments['ANSWERCMTTEXT'] == answer_cmt]['ANSWERCMTVOTECOUNTS'].tolist()[0]
      answer_cmt_dt = item.answer_comments[item.answer_comments['ANSWERCMTTEXT'] == answer_cmt]['ANSWERCMTDATETIME'].tolist()[0]
      if str(answer_cmt) != 'nan':
        answers_cmts.append({
          "answer_cmt": answer_cmt,
          "id": 1,
          "votes": answer_cmt_votes,
          "datetime": answer_cmt_dt
        })
        
    answers_cmts_joined = ' '.join([str(elem['answer_cmt']) for elem in answers_cmts])
    
    allCommentsGroupedCleanedDict[item.title] = {
      "comments": comments,
      "answers": answers,
      "answer_comments": answers_cmts,
      "comments_joined": comments_joined,
      "answers_joined": answers_joined,
      "answers_cmts_joined": answers_cmts_joined
    }

# print(allCommentsGroupedCleanedDict)

In [301]:
allCommentsGroupedCleanedDict[list(allCommentsGroupedCleanedDict.keys())[2]]['answer_comments']

[{'answer_cmt': 'Thanks, I found out what I missed and what was new.',
  'id': 1,
  'votes': "('',)",
  'datetime': 'Nov 4, 2022 at 15:18'},
 {'answer_cmt': 'i is not an index, but an element of list, so you cannot do list[i]',
  'id': 1,
  'votes': "('',)",
  'datetime': 'Nov 4, 2022 at 15:06'},
 {'answer_cmt': 'I think he wants them to be in the same order as the original list, not sorted numerically.',
  'id': 1,
  'votes': "('',)",
  'datetime': 'Nov 4, 2022 at 15:06'},
 {'answer_cmt': "I don't want to be sorted, I want it to be printed in the order of what's in the original list. I think I said something that could be misunderstood.",
  'id': 1,
  'votes': "('',)",
  'datetime': 'Nov 4, 2022 at 15:09'}]

In [292]:
from nltk.corpus import stopwords
import heapq
 
stopwords = set(stopwords.words('english'))

def frequencyCounting(text, verbose = False):
  translate_table = dict((ord(char), None) for char in punctuation if char not in [' ', '\t', ' ', '\r', ",", ';', '?', '!'])
  translate_table.update(dict((ord(char), None) for char in string.digits))

  compiledCommentsForFirstPost = text.translate(translate_table)

  word_frequencies = {}
  for word in word_tokenize(compiledCommentsForFirstPost):
      word = SnowballStemmer("english").stem(word)
      if word not in stopwords:
          if word not in punctuation:
            if word not in word_frequencies.keys():
                word_frequencies[word] = 1
            else:
                word_frequencies[word] += 1

  maximum_frequncy = max(word_frequencies.values(), default=0)
  for word in word_frequencies.keys():
      word_frequencies[word] = (word_frequencies[word]/maximum_frequncy)
  
  if(verbose):
    print(sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True))

  sentence_list = sent_tokenize(compiledCommentsForFirstPost)
  
  if(verbose):
    print('total sentences: ', len(sentence_list))

  sentence_scores = {}
  for sent in sentence_list:
      sentence = sent.split()
      for word in sentence:        
          if word.lower() in word_frequencies.keys():
              if sent not in sentence_scores.keys():
                  sentence_scores[sent] = word_frequencies[word.lower()]
              else:
                  sentence_scores[sent] += word_frequencies[word.lower()]
                  
  if(verbose):
    print('scores for sentences: ', sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True))
  
  return sentence_scores, sentence_list
  

results = []
for i in range(0, len(allCommentsGroupedCleanedDict)):
  comments_answers_answerscmt = allCommentsGroupedCleanedDict[list(allCommentsGroupedCleanedDict.keys())[i]]['comments_joined'] + ' ' + allCommentsGroupedCleanedDict[list(allCommentsGroupedCleanedDict.keys())[i]]['answers_joined'] + ' ' + allCommentsGroupedCleanedDict[list(allCommentsGroupedCleanedDict.keys())[i]]['answers_cmts_joined']
  
  if(len(comments_answers_answerscmt) > 0):
    sentence_scores, sentence_list = frequencyCounting(comments_answers_answerscmt, verbose=False)
    results.append({
      "title": list(allCommentsGroupedCleanedDict.keys())[i],
      "sentence_scores": sentence_scores,
      "sentence_list": sentence_list
    })
    summary_sentences = heapq.nlargest(5, sentence_scores, key=sentence_scores.get)
    print(summary_sentences)

results


['Why are checking states later than t in if item  terminated and item in temp and intitem t If I understand you correctly, you have to consider only state changes up to time t, dont you?', 'What is the problem with your code?']
['Remove steps jsondumpssteps, or move it after the loop']
['i is not a list index, its a list element listi is not the current element of the iteration Have you considered a list comprehension instead x for x in list if x   when you do for i in list i is an element of list, not an index, so you shouldnt do flistappendlisti but  flistappendi instead So on the first iteration, youre appending list, which is the last element of the list Also, list is a class in module builtins  it is a poor name for a variable for i in list iterates over the items in the list, not the indices Since the first four items in the list are negative numbers, when you use them as indices, you end up iterating through the last half of the list in reverse order before reaching zero and th

[{'title': 'how to search in a python list?',
  'sentence_scores': {'What is the problem with your code?': 0.5,
   'Why are checking states later than t in if item  terminated and item in temp and intitem t If I understand you correctly, you have to consider only state changes up to time t, dont you?': 6.0},
  'sentence_list': ['What is the problem with your code?',
   'Why are checking states later than t in if item  terminated and item in temp and intitem t If I understand you correctly, you have to consider only state changes up to time t, dont you?']},
 {'title': 'How to append json in Python List',
  'sentence_scores': {'Remove steps jsondumpssteps, or move it after the loop': 2.0},
  'sentence_list': ['Remove steps jsondumpssteps, or move it after the loop']},
 {'title': 'How do I add Python list in order?',
  'sentence_scores': {'i is not a list index, its a list element listi is not the current element of the iteration Have you considered a list comprehension instead x for x in

In [8]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoConfig
from scipy.special import softmax
import numpy as np
from array import array

tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
config = AutoConfig.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest")

list_of_sentences = [sentence[0] for sentence in sorted(sentence_scores.items(), key=lambda x: x[1], reverse=True)]

print(list_of_sentences)

output = model(**tokenizer(list_of_sentences, padding=True, truncation=True, return_tensors="pt"))

list_of_sentences_with_sentiment = []

for j in range(len(list_of_sentences)):
  scores = output[0][j].detach().numpy()
  scores = softmax(scores)
  ranking = np.argsort(scores)
  ranking = ranking[::-1]
  for i in range(scores.shape[0]):
      l = config.id2label[ranking[i]]
      s = scores[ranking[i]]

      if(list_of_sentences[j] not in [ sentence['sentence'] for sentence in list_of_sentences_with_sentiment]):
        list_of_sentences_with_sentiment.append(
          {
            'sentence': list_of_sentences[j],
            l:  np.round(float(s), 4),
          }
        )
      else:
        index =  [ sentence['sentence'] for sentence in list_of_sentences_with_sentiment].index(list_of_sentences[j])
        list_of_sentences_with_sentiment[index][l] = np.round(float(s), 4)
      
sorted_sentence_list_based_on_sentiment = sorted(list_of_sentences_with_sentiment, key=lambda x: x['positive'], reverse=True)
print(sorted_sentence_list_based_on_sentiment)

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


['stackoverflowcomquestions… this article explain the difference between an npflatten and a tfflatten use static vs dynamic ndarray I tried a test with the same data, using itertoolschainfromiterable   python mtimeit sfrom itertools import chain; l,,,,,, , , listchainfromiterablel It runs a bit more than twice as fast as the nested list comprehension thats the fastest of the alternatives shown here I found the syntax hard to understand until I realized you can think of it exactly like nested for loops for sublist in l for item in sublist yield item leaf for tree in forest for leaf in tree might be easier to comprehend and apply RobCrowell Same here To me the list comprehension one doesnt read right, something feels off about it  I always seem to get it wrong and end up googling To me this reads right leaf for leaf in tree for tree in forest I wish this is how it was I am sure I am missing something about the grammar here, and I would appreciate if anyone could point that out I kept loo

In [9]:
joined_sentences = ' '.join([sentence['sentence'] for sentence in sorted_sentence_list_based_on_sentiment])

print(joined_sentences)

Theres an indepth discussion of this here rightfootinblogspotcommoreonpythonflattenhtml, discussing several methods of flattening arbitrarily nested lists of lists An interesting read! Thats just a more complicated and a bit slower way of what ᴡʜᴀᴄᴋᴀᴍᴀᴅᴏᴏᴅʟᴇ already posted before I reinvented his proposal yesterday, so this approach seems quite popular these days ; Not quite wierdlist  , , , , , , , , ,   nicelist, , , , , , , , , ,  my code as one liner would be  flatlist  intereplace,replace, for e in strdeeplistsplit, You are indeed right , ᴡʜᴀᴄᴋᴀᴍᴀᴅᴏᴏᴅʟᴇs proposal wont work with multiple digit numbers, I also didnt test this before although it should be obvious You could simplify your code and write intestrip  for e in strdeeplistsplit, But Id suggest to stick with Deleets proposal for real use cases It doesnt contain hacky type transformations, its faster and more versatile because it naturally also handles lists with mixed types Unfortunately no But I saw this code recently here 

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
input_ids = tokenizer.encode(sorted_sentence_list_based_on_sentiment[0]['sentence'], return_tensors='pt')

summary_ids = model.generate(input_ids)

print(tokenizer.decode(summary_ids[0], skip_special_tokens=True))

/Users/shaun/miniforge3/lib/python3.9/site-packages/transformers/generation/utils.py:1387: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 142 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Theres an indepth discussion of this here rightfootinblogspotcommoreonpythonflattenhtml, discussing several methods of flattening arbitrarily nested lists of lists. An interesting read! Click here to read the rest of the article. Click here for the full article.


-Thursday- 
Rescrap all the data with codes structure in html 

-Friday-
Maybe we can think of summarizing before sentiment analysis
After sentiment analysis, we can judge if the post is indeed good enough.